### Import
---

In [12]:
import numpy as np


---
# 01 - Learning with errors problem (LWE)

Learning with errors (이하 LWE) 문제는 격자기반 동형암호에서 많이 사용하는 암호학적 어려움을 제공하는 문제이다.

암호학적 안정성을 검증하는 것은 매우 어려우니 일단 동형암호의 지식습득을 목표하고 있는 우리는 잠시 생략하도록 하자. ~~사실 나도 잘 모른다.~~

LWE 의 암호화(Encryption)는 다음과 같이 정의할 수 있다.

>$\textbf{Definition : LWE encryption}$
>
>무작위 값을 갖는 벡터 $\vec{a} \in \mathbb{Z}_q^n$ 와 비밀키 벡터 $\vec{s} \in \mathbb{Z}_q^n$ 그리고 작은 크기를 갖는 노이즈 $e$ 가 있다고 할 때 LWE 암호화는 다음과 같이 정의할 수 있다:
>
>\begin{align*}
>    \textsf{LWE.Enc}(m, \vec{s}) = (\vec{a}, b) \in \mathbb{Z}_q^{n+1} \text{, where } b = \langle \vec{a}, \vec{s} \rangle + m + e \text{ (mod } q) \in \mathbb{Z}_q.
>\end{align*}
>
>여기서 $m$ 은 암호화 하고자 하는 메세지를 의미하며, $\langle \cdot, \cdot \rangle$ 연산은 두 벡터의 내적(inner product)을 의미한다.

내적에 대해서 잘 모른다면 다음의 수식을 참고하자.

\begin{equation*}
    \langle \vec{a}, \vec{s} \rangle = \sum_{i=0}^{n-1} a_i \cdot s_i.
\end{equation*}

이제 이를 코드를 통해서 확인해보자.

In [13]:
# 정규분포(Gaussin distribution)를 따르는 난수 생성기
def discrete_gaussian(n, q, mean=0., std=3.2):
    coeffs = np.round(std * np.random.randn(n)) % q
    return np.array(coeffs, dtype = int)

# 균등분포(Uniform distribution)를 따르는 난수 생성기
def discrete_uniform(n, q, min=0., max=None):
    if max is None:
        max = q
    coeffs = np.random.randint(min, max, size=n)
    return np.array(coeffs, dtype = int)

In [14]:
def LWE_Enc(m:int, s:np.ndarray, n, q) -> tuple[np.ndarray, int]:
    a = discrete_uniform(n, q)
    e = discrete_gaussian(1, q)
    b = (np.sum(a * s) + m + e) % q   # <a,s> + m + e mod q

    return (a, b)

In [15]:
n = 16
q = 128
m = 20

s = discrete_gaussian(n, q)
ctxt = LWE_Enc(m, s, n, q)
a, b = ctxt

print("a         : ", a)
print("s         : ", s)
print("b         : ", b)
print("Message   : ", m)

a         :  [ 12  71  52  34  32  17  42 115 124  23  36  96  28 113  38  70]
s         :  [  1 125 124 124   3 125   0 127 126 122 126 127 125   1   2 123]
b         :  [13]
Message   :  20



---

위의 예제를 보면 메세지 $m$ 과 암호문 $b$ 는 완전히 다른 값인 것을 알 수 있다. 비밀키인 $\vec{s}$ 를 모를 경우 구하기가 매우 어려울 것이다.

암호화를 정의했으니 이제 복호화(Decryption)을 정의해보자.

>$\textbf{Definition : LWE decryption}$
>
>LWE 암호문 $\textsf{ct} = (\vec{a}, b)$ 와 비밀키 $\vec{s}$ 가 있다고 할 때, LWE 복호화는 다음과 같이 정의할 수 있다:
>
>\begin{align*}
>    \textsf{LWE.Dec}(\textsf{ct}, \vec{s}) = b - \langle \vec{a}, \vec{s} \rangle = m + e \approx m \in \mathbb{Z}_q.
>\end{align*}

자세히 보면 복호화 한 결과가 $m+e \approx m$ 으로 본래 메세지인 $m$ 과는 약간의 차이를 보이는 모습을 보인다.

이는 격자기반 동형암호의 가장 큰 특징 중 하나이며 본래의 메시지는 복원할 수 없다는 특징을 갖고있다. 

즉, 이러한 약간의 노이즈 $e$ 를 메세지에 영향을 받지 않도록 핸들링 하는 것이 매우 중요하다.

이는 후에 서술할 동형연산(homomorphic operation)에서 더욱 자세히 알아 볼 것이다.

다시 돌아와서 이를 코드로 확인해보자.

In [16]:
def LWE_Dec(ctxt:tuple[np.ndarray, int], s:np.ndarray, n, q) -> int:
    a,b = ctxt
    msg = (b - np.sum(a * s)) % q   # b - <a,s> mod q

    return int(msg[0])

In [17]:
n = 16
q = 128
m = 40

s = discrete_gaussian(n, q)
ctxt = LWE_Enc(m, s, n, q)

a, b = ctxt
ptxt = LWE_Dec(ctxt, s, n, q)

print("a               : ", a)
print("s               : ", s)
print("b               : ", b)
print("Message         : ", m)
print("Decrypted value : ", ptxt)
print("Noise           : ", np.abs(ptxt - m))

a               :  [  1  70  61  37 100  46  69  75  83 122  73  13  96 104 114  49]
s               :  [124 127   5   8   0   0   2 124   6 127   3 122   7 127 126   6]
b               :  [15]
Message         :  40
Decrypted value :  35
Noise           :  5



---

약간의 노이즈가 발생하긴 하지만 메세지와 거의 유사한 값을 복원한다는 것을 알 수 있다.

그럼 이제 동형암호의 정수인 암호문 간의 연산(동형연산, homomorphic operation)을 정의해보도록 하자.

>$\textbf{Definition : LWE homomorphic addition}$
>
>두 LWE 암호문 $\textsf{ct}_1 = (\vec{a}_1, b_1)$ 과 $\textsf{ct}_2 = (\vec{a}_2, b_2)$ 있다고 할 때 LWE 간의 덧셈은 다음과 같이 정의된다.
>
>\begin{align*}
>    \textsf{LWE.Add}(\textsf{ct}_1, \textsf{ct}_2) = (\vec{a}_1 +' \vec{a}_2, b_1 + b_2) \in \mathbb{Z}_q^{n+1} = \textsf{ct}_\text{add}.
>\end{align*}
>
>여기서 $+'$ 은 두 벡터간 원소들의 element-wise addition 이다.
>
>비밀키를 $\vec{s}$ 라 하고, 각 암호문이 갖고 있던 메세지를 $m_1, m_2$ 라 할 때 덧셈의 결과는 $\textsf{LWE.Dec}(\textsf{ct}_\text{add}, \vec{s}) \approx m_1 + m_2$ 이다.

$\textsf{LWE.Dec}(\textsf{ct}_\text{add}, \vec{s}) \approx m_1 + m_2$ 가 성립하는지 확인해보자.

>$\textbf{Correctness : LWE homomorphic addition}$
>
>$\textsf{ct}_\text{add} = (\vec{a'}, b')$ 이라고 하자. 그럼 다음의 수식이 성립한다.
>
>\begin{align*}
>
>    \textsf{LWE.Dec}(\textsf{ct}_\text{add}, \vec{s}) &= b' - \langle \vec{a'}, \vec{s} \rangle \\
>                                                      &= b' - \langle \vec{a}_1 +' \vec{a}_2, \vec{s} \rangle \\
>                                                      &= b' - (\langle \vec{a}_1, \vec{s} \rangle + \langle \vec{a}_2, \vec{s} \rangle) \\
>                                                      &= (b_1 + b_2) - (\langle \vec{a}_1, \vec{s} \rangle + \langle \vec{a}_2, \vec{s} \rangle) \\
>                                                      &= (b_1 - \langle \vec{a}_1, \vec{s} \rangle) + (b_2 - \langle \vec{a}_2, \vec{s} \rangle) \\
>                                                      &= m_1 + e_1 + m_2 + e_2 \\
>                                                      &= (m_1 + m_2) + (e_1 + e_2) \approx m_1 + m_2
>
>\end{align*}

코드로 확인해보자!

In [18]:
def LWE_add(ct1:tuple[np.ndarray, int], ct2:tuple[np.ndarray, int]) -> tuple[np.ndarray, int]:
    a_1, b_1 = ct1
    a_2, b_2 = ct2

    a_add = a_1 + a_2
    b_add = b_1 + b_2

    return(a_add, b_add)
    

In [19]:
n  = 16
q  = 128
m1 = 40
m2 = 20

s = discrete_gaussian(n, q)

ct1  = LWE_Enc(m1, s, n, q)
ct2  = LWE_Enc(m2, s, n, q)

ct_add = LWE_add(ct1, ct2)

pt_add = LWE_Dec(ct_add, s, n, q)

print("Message         : ", m1 + m2)
print("Decrypted value : ", pt_add)
print("Noise           : ", np.abs(pt_add - (m1 + m2)))

Message         :  60
Decrypted value :  61
Noise           :  1



---

역시나 기대하는 값과 약간의 차이를 보이기는 하지만 유사한 값을 복원할 수 있음을 알 수 있다. 

그러나 동형암호가 만능은 아니다. 결과로 복원되는 값은 $(m_1 + m_2) + (e_1 + e_2) \approx m_1 + m_2$ 이다. 즉, 동형연산을 수행했더니 노이즈가 커져버린 것이다.

다시말해 일정 수준 이상 연산을 수행하게 되면, 노이즈가 증폭되어서 값을 알아볼 수 없게 된다는 것이다.

다음 예제를 확인해보자.

In [20]:
n   = 16
q   = 128
m   = 0
one = 1

ct     = LWE_Enc(m, s, n, q)
ct_one = LWE_Enc(one, s, n, q)

for i in range(50):
    ct = LWE_add(ct, ct_one)
    m += one

ptxt = LWE_Dec(ct, s, n, q)

print("Message         : ", m)
print("Decrypted value : ", ptxt)
print("Noise           : ", np.abs(ptxt - m))

Message         :  50
Decrypted value :  22
Noise           :  28


위와 같이 복호화 한 값이 이전과는 달리 큰 오차를 보이는 경우가 다수 발생한다.

실제 동형암호를 잘 활용하기 위해서는 이러한 노이즈를 잘 핸들링 해야한다. 그렇지 않다면 동형연산의 횟수에 제한이 생겨 활용할 수 있는 영역이 제한되어 버린다.

이러한 한계를 극복하기 위해 Gentry 는 Bootstrapping 이라는 개념을 도입하여 동형연산을 이용하여 암호문 내의 노이즈를 줄이는 컨셉을 통해 동형암호의 활용성에 크게 기여했다.

* <a href = "https://dl.acm.org/doi/10.1145/1536414.1536440">Fully homomorphic encryption using ideal lattices </a>



---

중요한 동형연산은 잘 마무리 했다. 이번에는 비교적 쉬운 평문 덧셈을 정의해보자.

>$\textbf{Definition : LWE plaintext addition}$
>
>LWE 암호문 $\textsf{ct} = (\vec{a}, b)$ 과 평문 $t$ 있다고 할 때 암호문과 평문간의 덧셈은 다음과 같이 정의된다.
>
>\begin{align*}
>    \textsf{LWE.Add}(\textsf{ct},t) = (\vec{a} , b + t) \in \mathbb{Z}_q^{n+1} = \textsf{ct}_\text{add}.
>\end{align*}
>
>
>비밀키를 $\vec{s}$ 라 하고, 암호문이 갖고 있던 메세지를 $m$ 이라 할 때 덧셈의 결과는 $\textsf{LWE.Dec}(\textsf{ct}_\text{add}, \vec{s}) \approx m + t$ 이다.

>$\textbf{Correctness : LWE plaintext addition}$
>
>$\textsf{ct}_\text{add} = (\vec{a'}, b')$ 이라고 하자. 그럼 다음의 수식이 성립한다.
>
>\begin{align*}
>
>    \textsf{LWE.Dec}(\textsf{ct}_\text{add}, \vec{s}) &= b' - \langle \vec{a'}, \vec{s} \rangle \\
>                                                      &= (b+t) - \langle \vec{a'}, \vec{s} \rangle \\
>                                                      &= b - \langle \vec{a}, \vec{s} \rangle + t \\
>                                                      &= m + t + e \approx m + t
>
>\end{align*}

이제 코드를 통해 확인해보자!

In [21]:
def LWE_add_ptxt(ct:tuple[np.ndarray, int], t:int) -> tuple[np.ndarray, int]:
    a, b = ct
    return(a, b + t)

In [22]:
n  = 16
q  = 128
m  = 40
t  = 10

s = discrete_gaussian(n, q)

ct  = LWE_Enc(m, s, n, q)
ct_add = LWE_add_ptxt(ct, t)

pt_add = LWE_Dec(ct_add, s, n, q)

print("Message         : ", m + t)
print("Decrypted value : ", pt_add)
print("Noise           : ", np.abs(pt_add - (m + t)))

Message         :  50
Decrypted value :  47
Noise           :  3



---

지금까지는 LWE 에서 사용하는 암호문 간의 덧셈과 평문 덧셈을 정의했다.

우리가 주로 사용하는 곱셈은 일반적으로 LWE 에서 사용되지 않으므로 생략하고 다음 RLWE 를 설명할 때 자세히 다루도록 한다.

---

## Code Implementation

`note_include/elem/LWE.py` 를 확인하면 위에서 정의한 연산들이 있는 LWE 구현을 확인할 수 있다.